In [31]:
import numpy as np
import matplotlib.pyplot as plt


In [32]:
def interpolate_temperature(
    resistance: float,
    resistance_interval: tuple[float, float],
    temperature_interval: tuple[float, float],
) -> float:
    delta_R = resistance_interval[1] - resistance_interval[0]
    delta_T = temperature_interval[1] - temperature_interval[0]
    return temperature_interval[0] + delta_T / delta_R * (
        resistance - resistance_interval[0]
    )


def get_resistance_interval(
    resistance: float, resistance_array: np.ndarray
) -> tuple[int, int]:
    for index, i in enumerate(resistance_array):
        if i >= resistance:
            return index, index + 1
    return 0, 1


In [33]:
def get_temperature(
    R: np.ndarray, reference_R: np.ndarray, reference_T: np.ndarray
) -> np.ndarray:
    temperature = np.zeros_like(R)
    for index, resistance in enumerate(R):
        lower, upper = get_resistance_interval(resistance, reference_R)
        temperature[index] = interpolate_temperature(
            resistance,
            (reference_R[lower], reference_R[upper]),
            (reference_T[lower], reference_T[upper]),
        )
    return temperature


In [34]:
def calculate_constant(
    sensor_voltage: np.ndarray, temperature: np.ndarray
) -> np.ndarray:
    return temperature**4 / sensor_voltage


In [35]:
def get_epsilon(
    a: np.ndarray, sensor_voltage: np.ndarray, temperature: np.ndarray
) -> np.ndarray:
    return a * sensor_voltage / temperature**4


In [36]:
def get_results(a: np.ndarray, voltage: np.ndarray, temperature: np.ndarray) -> tuple[float, float]:
    epsilon = get_epsilon(a, voltage, temperature)
    epsilon_avg = np.average(epsilon)
    epsilon_std = np.std(epsilon, ddof=1)
    return float(epsilon_avg), float(epsilon_std)

In [37]:
data = np.loadtxt("results.csv", unpack=True, delimiter=",", skiprows=1)
resistance = data[0] * 1000
black_voltage, grey_voltage, mirror_voltage, white_voltage = data[1:] / 1000

reference_R, reference_temp = np.loadtxt(
    "resistance_temperature_table.csv", unpack=True, delimiter=",", skiprows=1
)

In [38]:
temperature = get_temperature(resistance, reference_R, reference_temp)
A = calculate_constant(black_voltage, temperature)


### Grey

In [39]:
grey_epsilon = get_results(A, grey_voltage, temperature)
print(grey_epsilon)


(0.3570214635578706, 0.004509384063104793)


### Mirror

In [40]:
mirror_epsilon = get_results(A, mirror_voltage, temperature)
print(mirror_epsilon)


(0.05719267597982905, 0.009300257242144389)


### White

In [41]:
white_epsilon = get_results(A, white_voltage, temperature)
print(white_epsilon)


(0.9604884445339257, 0.0059874761207086)
